# Installs

In [ ]:
!pip install wandb -q

### Levenshtein

This may take a while

In [ ]:
!pip install python-Levenshtein
!git clone --recursive https://github.com/parlance/ctcdecode.git
!pip install wget
%cd ctcdecode
!pip install .
%cd ..

!pip install torchsummaryX

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
fatal: destination path 'ctcdecode' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/content/ctcdecode
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/ctcdecode
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for ctcdecode: filename=ctcdecode-1.0.3-cp37-cp37m-linux_x86_64.whl size=13405118 sha256=4d67ba54cdc5d3adf79a7649095b9de50694dcb37e972b44f464301b35dd6e87
  Stor

/content
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## imports

In [ ]:
import torch
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

import torchaudio.transforms as tat

from sklearn.metrics import accuracy_score
import gc
import wandb
import time

import zipfile
import pandas as pd
from tqdm import tqdm
import os
import datetime


# imports for decoding and distance calculation
import ctcdecode
import Levenshtein
from ctcdecode import CTCBeamDecoder

import warnings
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


# Kaggle Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"ruidichang","key":"ec93dcec34e885b48bb96b02e9b79b80"}') # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached kaggle-1.5.8-py3-none-any.whl
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.8
    Uninstalling kaggle-1.5.8:
      Successfully uninstalled kaggle-1.5.8
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
'''
This will take a couple minutes, but you should see at least the following:
11-785-f22-hw3p2.zip  ctcdecode  hw3p2
'''

!kaggle competitions download -c 11-785-f22-hw3p2
!unzip -q 11-785-f22-hw3p2.zip

!ls

11-785-f22-hw3p2.zip: Skipping, found more recently modified local copy (use --force to force download)
replace hw3p2/dev-clean/mfcc/0084_121123_000.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: 11-785-f22-hw3p2.zip  ctcdecode  drive	hw3p2  sample_data


# Dataset and Dataloader

## Model Config

In [ ]:
# ARPABET PHONEME MAPPING
# DO NOT CHANGE
# This overwrites the phonetics.py file.

CMUdict_ARPAbet = {
    "" : " ", # BLANK TOKEN
    "[SIL]": "-", "NG": "G", "F" : "f", "M" : "m", "AE": "@", 
    "R"    : "r", "UW": "u", "N" : "n", "IY": "i", "AW": "W", 
    "V"    : "v", "UH": "U", "OW": "o", "AA": "a", "ER": "R", 
    "HH"   : "h", "Z" : "z", "K" : "k", "CH": "C", "W" : "w", 
    "EY"   : "e", "ZH": "Z", "T" : "t", "EH": "E", "Y" : "y", 
    "AH"   : "A", "B" : "b", "P" : "p", "TH": "T", "DH": "D", 
    "AO"   : "c", "G" : "g", "L" : "l", "JH": "j", "OY": "O", 
    "SH"   : "S", "D" : "d", "AY": "Y", "S" : "s", "IH": "I",
    "[SOS]": "[SOS]", "[EOS]": "[EOS]"}

CMUdict = list(CMUdict_ARPAbet.keys())
ARPAbet = list(CMUdict_ARPAbet.values())


PHONEMES = CMUdict
mapping = CMUdict_ARPAbet
LABELS = ARPAbet

In [ ]:
OUT_SIZE = len(LABELS)
OUT_SIZE

43

In [ ]:
config = {
    "beam_width" : 30,
    "lr" : 0.00005,
    "epochs" : 50,
    "batch_size" : 64
    } # Feel free to add more items here

In [ ]:
# You might want to play around with the mapping as a sanity check here

## Train Data

In [ ]:
class AudioDataset(torch.utils.data.Dataset):

    # For this homework, we give you full flexibility to design your data set class.
    # Hint: The data from HW1 is very similar to this HW

    #TODO
    def __init__(self, PHONEMES): 
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''

        # Load the directory and all files in them
        path = '/content/hw3p2/train-clean-360/'
        self.mfcc_dir = os.path.join(path,'mfcc')
        self.transcript_dir = os.path.join(path,'transcript/raw')
        self.mfcc_files = sorted(os.listdir(self.mfcc_dir))
        self.transcript_files = sorted(os.listdir(self.transcript_dir))

        self.phonemes = PHONEMES
        self.mfccs, self.transcripts = [], []

        assert len(self.mfcc_files) == len(self.transcript_files) # Making sure that we have the same no. of mfcc and transcripts

        #TODO
        # WHAT SHOULD THE LENGTH OF THE DATASET BE?
        self.length = len(self.mfcc_files)

        #TODO
        # CREATE AN ARRAY OF ALL FEATUERS AND LABELS
        # WHAT NORMALIZATION TECHNIQUE DID YOU USE IN HW1? CAN WE USE IT HERE?
        '''
        You may decide to do this in __getitem__ if you wish.
        However, doing this here will make the __init__ function take the load of
        loading the data, and shift it away from training.
        '''

        for i in range(0, self.length):
        #   Load a single mfcc
            mfcc = np.load(os.path.join(self.mfcc_dir,self.mfcc_files[i]))
        #   Optionally do Cepstral Normalization of mfcc
        #   Load the corresponding transcript
            transcript = np.load(os.path.join(self.transcript_dir,self.transcript_files[i]))
            transcript = transcript[1:-1] # Remove [SOS] and [EOS] from the transcript (Is there an efficient way to do this 
            # without traversing through the transcript?)
        
        #TODO
        # HOW CAN WE REPRESENT PHONEMES? CAN WE CREATE A MAPPING FOR THEM?
        # # HINT: TENSORS CANNOT STORE NON-NUMERICAL VALUES OR STRINGS  
        #   Append each mfcc to self.mfcc, transcript to self.transcript
            self.mfccs.append(mfcc)
            self.transcripts.append([self.phonemes.index(t) for t in transcript])


        # Load the directory and all files in them
        path = '/content/hw3p2/train-clean-100/'
        self.mfcc_dir = os.path.join(path,'mfcc')
        self.transcript_dir = os.path.join(path,'transcript/raw')
        self.mfcc_files = sorted(os.listdir(self.mfcc_dir))
        self.transcript_files = sorted(os.listdir(self.transcript_dir))

        assert len(self.mfcc_files) == len(self.transcript_files) # Making sure that we have the same no. of mfcc and transcripts

        #TODO
        # WHAT SHOULD THE LENGTH OF THE DATASET BE?
        self.length = len(self.mfcc_files)

        #TODO
        # CREATE AN ARRAY OF ALL FEATUERS AND LABELS
        # WHAT NORMALIZATION TECHNIQUE DID YOU USE IN HW1? CAN WE USE IT HERE?
        '''
        You may decide to do this in __getitem__ if you wish.
        However, doing this here will make the __init__ function take the load of
        loading the data, and shift it away from training.
        '''

        for i in range(0, self.length):
        #   Load a single mfcc
            mfcc = np.load(os.path.join(self.mfcc_dir,self.mfcc_files[i]))
        #   Optionally do Cepstral Normalization of mfcc
        #   Load the corresponding transcript
            transcript = np.load(os.path.join(self.transcript_dir,self.transcript_files[i]))
            transcript = transcript[1:-1] # Remove [SOS] and [EOS] from the transcript (Is there an efficient way to do this 
            # without traversing through the transcript?)
        
        #TODO
        # HOW CAN WE REPRESENT PHONEMES? CAN WE CREATE A MAPPING FOR THEM?
        # # HINT: TENSORS CANNOT STORE NON-NUMERICAL VALUES OR STRINGS  
        #   Append each mfcc to self.mfcc, transcript to self.transcript
            self.mfccs.append(mfcc)
            self.transcripts.append([self.phonemes.index(t) for t in transcript])



    def __len__(self):
        
        '''
        TODO: What do we return here?
        '''
        return len(self.mfccs)

    def __getitem__(self, ind):
        '''
        TODO: RETURN THE MFCC COEFFICIENTS AND ITS CORRESPONDING LABELS

        If you didn't do the loading and processing of the data in __init__,
        do that here.

        Once done, return a tuple of features and labels.
        '''

        mfcc = torch.tensor(self.mfccs[ind])
        transcript = torch.tensor(self.transcripts[ind]) 
        return mfcc, transcript


    def collate_fn(batch):
        '''
        TODO:
        1.  Extract the features and labels from 'batch'
        2.  We will additionally need to pad both features and labels,
            look at pytorch's docs for pad_sequence
        3.  This is a good place to perform transforms, if you so wish. 
            Performing them on batches will speed the process up a bit.
        4.  Return batch of features, labels, lenghts of features, 
            and lengths of labels.
        '''
        # batch of input mfcc coefficients
        batch_mfcc = [mfcc for mfcc,transcript in batch]
        # batch of output phonemes
        batch_transcript = [transcript for mfcc,transcript in batch]

        # HINT: CHECK OUT -> pad_sequence (imported above)
        # Also be sure to check the input format (batch_first)
        batch_mfcc_pad = pad_sequence(batch_mfcc)
        lengths_mfcc = [sample.shape[0] for sample in batch_mfcc]

        batch_transcript_pad = pad_sequence(batch_transcript, batch_first=True)
        lengths_transcript = [sample.shape[0] for sample in batch_transcript]

        # You may apply some transformation, Time and Frequency masking, here in the collate function;
        # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
        #                  -> Would we apply transformation on the validation set as well?
        #                  -> Is the order of axes / dimensions as expected for the transform functions?
        
        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return batch_mfcc_pad, batch_transcript_pad, torch.tensor(lengths_mfcc), torch.tensor(lengths_transcript)

## Validation Data

In [ ]:
class AudioDatasetValidation(torch.utils.data.Dataset):

    # For this homework, we give you full flexibility to design your data set class.
    # Hint: The data from HW1 is very similar to this HW

    #TODO
    def __init__(self,PHONEMES): 
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''

        # Load the directory and all files in them
        path = '/content/hw3p2/dev-clean/'
        self.mfcc_dir = os.path.join(path,'mfcc')
        self.transcript_dir = os.path.join(path,'transcript/raw')
        self.mfcc_files = sorted(os.listdir(self.mfcc_dir))
        self.transcript_files = sorted(os.listdir(self.transcript_dir))

        self.phonemes = PHONEMES
        self.mfccs, self.transcripts = [], []

        assert len(self.mfcc_files) == len(self.transcript_files) # Making sure that we have the same no. of mfcc and transcripts

        #TODO
        # WHAT SHOULD THE LENGTH OF THE DATASET BE?
        self.length = len(self.mfcc_files)

        #TODO
        # CREATE AN ARRAY OF ALL FEATUERS AND LABELS
        # WHAT NORMALIZATION TECHNIQUE DID YOU USE IN HW1? CAN WE USE IT HERE?
        '''
        You may decide to do this in __getitem__ if you wish.
        However, doing this here will make the __init__ function take the load of
        loading the data, and shift it away from training.
        '''
        
        for i in range(0, self.length):
        #   Load a single mfcc
            mfcc = np.load(os.path.join(self.mfcc_dir,self.mfcc_files[i]))
        #   Optionally do Cepstral Normalization of mfcc
        #   Load the corresponding transcript
            transcript = np.load(os.path.join(self.transcript_dir,self.transcript_files[i]))
            transcript = transcript[1:len(transcript)-1] # Remove [SOS] and [EOS] from the transcript (Is there an efficient way to do this 
            # without traversing through the transcript?)
        #   Append each mfcc to self.mfcc, transcript to self.transcript
            self.mfccs.append(mfcc)
            self.transcripts.append([self.phonemes.index(t) for t in transcript])

        #TODO
        # HOW CAN WE REPRESENT PHONEMES? CAN WE CREATE A MAPPING FOR THEM?
        # HINT: TENSORS CANNOT STORE NON-NUMERICAL VALUES OR STRINGS
        # self.transcripts = torch.LongTensor(np.asarray([self.phonemes.index(t) for t in self.transcripts[1:-1]]))

    def __len__(self):
        
        '''
        TODO: What do we return here?
        '''
        return len(self.mfccs)

    def __getitem__(self, ind):
        '''
        TODO: RETURN THE MFCC COEFFICIENTS AND ITS CORRESPONDING LABELS

        If you didn't do the loading and processing of the data in __init__,
        do that here.

        Once done, return a tuple of features and labels.
        '''

        mfcc = torch.tensor(self.mfccs[ind])
        transcript = torch.tensor(self.transcripts[ind]) 
        return mfcc, transcript


    def collate_fn(batch):
        '''
        TODO:
        1.  Extract the features and labels from 'batch'
        2.  We will additionally need to pad both features and labels,
            look at pytorch's docs for pad_sequence
        3.  This is a good place to perform transforms, if you so wish. 
            Performing them on batches will speed the process up a bit.
        4.  Return batch of features, labels, lenghts of features, 
            and lengths of labels.
        '''
        # batch of input mfcc coefficients
        batch_mfcc = [mfcc for mfcc,transcript in batch]
        # batch of output phonemes
        batch_transcript = [transcript for mfcc,transcript in batch]

        # HINT: CHECK OUT -> pad_sequence (imported above)
        # Also be sure to check the input format (batch_first)
        batch_mfcc_pad = pad_sequence(batch_mfcc)
        lengths_mfcc = [sample.shape[0] for sample in batch_mfcc]

        batch_transcript_pad = pad_sequence(batch_transcript, batch_first=True)
        lengths_transcript = [sample.shape[0] for sample in batch_transcript]

        # You may apply some transformation, Time and Frequency masking, here in the collate function;
        # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
        #                  -> Would we apply transformation on the validation set as well?
        #                  -> Is the order of axes / dimensions as expected for the transform functions?
        
        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return batch_mfcc_pad, batch_transcript_pad, torch.tensor(lengths_mfcc), torch.tensor(lengths_transcript)

## Test Data

In [ ]:
# Test Dataloader
#TODO
class AudioDatasetTest(torch.utils.data.Dataset):

    # For this homework, we give you full flexibility to design your data set class.
    # Hint: The data from HW1 is very similar to this HW

    #TODO
    def __init__(self): 
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''

        # Load the directory and all files in them
        path = '/content/hw3p2/test-clean/'
        self.mfcc_dir = os.path.join(path,'mfcc')
        self.mfcc_files = sorted(os.listdir(self.mfcc_dir))
        # self.mfcc_files = os.listdir(self.mfcc_dir)

        self.mfccs = []

        #TODO
        # WHAT SHOULD THE LENGTH OF THE DATASET BE?
        self.length = len(self.mfcc_files)

        #TODO
        # CREATE AN ARRAY OF ALL FEATUERS AND LABELS
        # WHAT NORMALIZATION TECHNIQUE DID YOU USE IN HW1? CAN WE USE IT HERE?
        '''
        You may decide to do this in __getitem__ if you wish.
        However, doing this here will make the __init__ function take the load of
        loading the data, and shift it away from training.
        '''
        
        for i in range(0, self.length):
        #   Load a single mfcc
            mfcc = np.load(os.path.join(self.mfcc_dir,self.mfcc_files[i]))
        #   Optionally do Cepstral Normalization of mfcc
        #   Append each mfcc to self.mfcc, transcript to self.transcript
            self.mfccs.append(mfcc)

    def __len__(self):
        
        '''
        TODO: What do we return here?
        '''
        return len(self.mfccs)

    def __getitem__(self, ind):
        '''
        TODO: RETURN THE MFCC COEFFICIENTS AND ITS CORRESPONDING LABELS

        If you didn't do the loading and processing of the data in __init__,
        do that here.

        Once done, return a tuple of features and labels.
        '''

        mfcc = torch.tensor(self.mfccs[ind])
        return mfcc


    def collate_fn(batch):
        '''
        TODO:
        1.  Extract the features and labels from 'batch'
        2.  We will additionally need to pad both features and labels,
            look at pytorch's docs for pad_sequence
        3.  This is a good place to perform transforms, if you so wish. 
            Performing them on batches will speed the process up a bit.
        4.  Return batch of features, labels, lenghts of features, 
            and lengths of labels.
        '''
        # batch of input mfcc coefficients
        batch_mfcc = [mfcc for mfcc in batch]
        
        # HINT: CHECK OUT -> pad_sequence (imported above)
        # Also be sure to check the input format (batch_first)
        batch_mfcc_pad = pad_sequence(batch_mfcc)
        lengths_mfcc = [sample.shape[0] for sample in batch_mfcc]

        # You may apply some transformation, Time and Frequency masking, here in the collate function;
        # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
        #                  -> Would we apply transformation on the validation set as well?
        #                  -> Is the order of axes / dimensions as expected for the transform functions?
        
        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return batch_mfcc_pad, torch.tensor(lengths_mfcc)

## Data - Hyperparameters

In [ ]:
transforms = [] # set of tranformations
# You may pass this as a parameter to the dataset class above
# This will help modularize your implementation

root = '/content/hw3p2' 

## Data loaders

In [ ]:
# get me RAMMM!!!! 
import gc 
gc.collect()

195

In [ ]:
# Create objects for the dataset class
train_data = AudioDataset(PHONEMES) #TODO
val_data = AudioDatasetValidation(PHONEMES) # TODO : You can either use the same class with some modifications or make a new one :)

# Do NOT forget to pass in the collate function as parameter while creating the dataloader
train_loader = DataLoader(train_data, num_workers= 2,
                                           batch_size=config['batch_size'], pin_memory= True,
                                           shuffle= True,collate_fn=AudioDataset.collate_fn)
val_loader = DataLoader(val_data, num_workers= 2,
                                           batch_size=config['batch_size'], pin_memory= True,
                                           shuffle= True, collate_fn=AudioDatasetValidation.collate_fn)

print("Batch size: ", config['batch_size'])
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))

Batch size:  64
Train dataset samples = 132553, batches = 2072
Val dataset samples = 2703, batches = 43


In [ ]:
test_data = AudioDatasetTest() #TODO
test_loader = DataLoader(test_data, num_workers= 2,
                                           batch_size=config['batch_size'], pin_memory= True,
                                           shuffle= False,collate_fn=AudioDatasetTest.collate_fn)
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Test dataset samples = 2620, batches = 41


In [ ]:
# sanity check
for data in train_loader:
    x, y, lx, ly = data
    print(x.shape, y.shape, lx.shape, ly.shape)
    break 

torch.Size([1630, 64, 15]) torch.Size([64, 214]) torch.Size([64]) torch.Size([64])


In [ ]:
# sanity check
for data in test_loader:
    t, lt = data
    print(t.shape, lt.shape)
    break 

torch.Size([825, 64, 15]) torch.Size([64])


## wandb

You will need to fetch your api key from wandb.ai

In [ ]:
wandb.login(key="f12c6cea0bcd560db008417ca06230259ed77e61") #API Key is in your wandb account, under settings (wandb.ai/settings)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
run = wandb.init(
    name = "early-submission", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # run_id = ### Insert specific run id here if you want to resume a previous run
    # resume = "must" ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw3p2-ablations", ### Project should be created in your wandb account 
    config = config ### Wandb Config for your run
)

wandb: Currently logged in as: ruidichang (dlp_team_2022). Use `wandb login --relogin` to force relogin


## Advanced

In [ ]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.cnn = torch.nn.Sequential(
            nn.BatchNorm1d(15), 
            nn.Conv1d(15, 256, kernel_size=3, padding=1),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True)
          )
        self.lstm = nn.LSTM(input_size=256, hidden_size=256, num_layers=3, bidirectional=True, dropout=0.3)
        self.fc1 = nn.Linear(512, 43)
    
    def forward(self, X, lengths):
        X = self.cnn(X.permute(1, 2, 0).contiguous()).permute(2, 0, 1)
        packed_X = pack_padded_sequence(X, lengths, enforce_sorted=False)
        packed_out = self.lstm(packed_X)[0]
        out, out_lens = pad_packed_sequence(packed_out)
        out = self.fc1(out).log_softmax(2)
        return out, out_lens

## INIT

In [ ]:
model = Network().to(device)
summary(model, x.to(device), lx) # x and lx come from the sanity check above :)

                     Kernel Shape     Output Shape     Params  Mult-Adds
Layer                                                                   
0_cnn.BatchNorm1d_0          [15]   [64, 15, 1630]       30.0       15.0
1_cnn.Conv1d_1       [15, 256, 3]  [64, 256, 1630]    11.776k   18.7776M
2_cnn.BatchNorm1d_2         [256]  [64, 256, 1630]      512.0      256.0
3_cnn.ReLU_3                    -  [64, 256, 1630]          -          -
4_lstm                          -     [77396, 512]  4.206592M  4.194304M
5_fc1                   [512, 43]   [1630, 64, 43]    22.059k    22.016k
-------------------------------------------------------------------------
                          Totals
Total params           4.240969M
Trainable params       4.240969M
Non-trainable params         0.0
Mult-Adds             22.994191M


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_cnn.BatchNorm1d_0,[15],"[64, 15, 1630]",30.0,15.0
1_cnn.Conv1d_1,"[15, 256, 3]","[64, 256, 1630]",11776.0,18777600.0
2_cnn.BatchNorm1d_2,[256],"[64, 256, 1630]",512.0,256.0
3_cnn.ReLU_3,-,"[64, 256, 1630]",NaN,NaN
4_lstm,-,"[77396, 512]",4206592.0,4194304.0
5_fc1,"[512, 43]","[1630, 64, 43]",22059.0,22016.0


# Training Config

In [ ]:
criterion = nn.CTCLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.00005, weight_decay=5e-6)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')
decoder = CTCBeamDecoder(PHONEMES, beam_width = 10, num_processes=os.cpu_count(), log_probs_input=True)# Mixed Precision, if you need it
scaler = torch.cuda.amp.GradScaler()

### Levenshtein

In [ ]:
# Use debug = True to see debug outputs
def calculate_levenshtein(h, y, lh, ly, decoder, labels, debug = False):

    if debug:
        print(f"\n----- IN LEVENSHTEIN -----\n")
        # Add any other debug statements as you may need
        # you may want to use debug in several places in this function
    h = h.permute(1,0,2)    
    # TODO: look at docs for CTC.decoder and find out what is returned here
    beam_result, beam_scores, timesteps, out_len = decoder.decode(h, seq_lens = lh)

    batch_size = y.shape[0] # TODO

    distance = 0 # Initialize the distance to be 0 initially

    for i in range(batch_size): 
        h_sliced = beam_result[i][0][:out_len[i][0]] # TODO: Get the output as a sequence of numbers from beam_results
        # Remember that h is padded to the max sequence length and lh contains lengths of individual sequences
        # Same goes for beam_results and out_lens
        # You do not require the padded portion of beam_results - you need to slice it with out_lens 
        # If it is confusing, print out the shapes of all the variables and try to understand
        h_string = "".join([str(labels[hh]) for hh in h_sliced])  # TODO: MAP the sequence of numbers to its corresponding characters with PHONEME_MAP and merge everything as a single string

        y_sliced = y[i,:ly[i]] # TODO: Do the same for y - slice off the padding with ly
        y_string = "".join([str(labels[yy])for yy in y_sliced])  # TODO: MAP the sequence of numbers to its corresponding characters with PHONEME_MAP and merge everything as a single string
        
        distance += Levenshtein.distance(h_string, y_string)


    distance /= batch_size # TODO: Uncomment this, but think about why we are doing this

    return distance

In [ ]:
# ANOTEHR SANITY CHECK

with torch.no_grad():
  for i, data in enumerate(train_loader):
      
      #TODO: 
      # Follow the following steps, and 
      # Add some print statements here for sanity checking
      
      #1. What values are you returning from the collate function
      #2. Move the features and target to <DEVICE>
      #3. Print the shapes of each to get a fair understanding 
      #4. Pass the inputs to the model
            # Think of the following before you implement:
            # 4.1 What will be the input to your model?
            # 4.2 What would the model output?
            # 4.3 Print the shapes of the output to get a fair understanding 

      # Calculate loss: https://pytorch.org/docs/stable/generated/torch.nn.CTCLoss.html
      # Calculating the loss is not straightforward. Check the input format of each parameter
      x = x.to(device)
      h, lh = model(x, lx)
      loss = criterion(h,y,lh,ly) # What goes in here?
      print(f"loss: {loss}")
      print(h.size())
      distance = calculate_levenshtein(h, y, lh, ly, decoder, PHONEMES)
      print(f"lev-distance: {distance}")

      break # one iteration is enough

loss: 0.20107023417949677
torch.Size([1630, 64, 43])
lev-distance: 11.984375


# Training

In [ ]:
def train(epoch, model, train_loader, optimizer, criterion):
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=True, position=0, desc=f'Train epoch: {epoch+1}') 

    model.train()

    total_loss, total = 0, 0

    log_interval = 40

    for batch, (data) in enumerate(train_loader):

        optimizer.zero_grad() 

        x, y, lx, ly = data
        total += len(y)

        x = x.to(device)
        outputs, length = model(x, lx)

        loss = criterion(outputs, y, length, ly)

        batch_bar.set_postfix(
            loss = f"{total_loss/ (batch+1):.4f}",
            lr = f"{optimizer.param_groups[0]['lr']}"
        )
        total_loss += loss.item()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.25)

        batch_bar.update()
    batch_bar.close()
    return total_loss/(batch+1)

In [ ]:
def validate(model, val_loader, criterion):
    model.eval()
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, leave=True, position=0, desc=f'Validation epoch: {epoch+1}') 

    total_loss, total_num = 0, 0
    total_dist = 0

    with torch.no_grad():
        for batch, (data) in enumerate(val_loader):
            x, y, lx, ly = data

            x = x.to(device)
            outputs, length = model(x, lx)

            loss = criterion(outputs, y, length, ly)
            total_loss += loss.item()

            total_dist += calculate_levenshtein(outputs, y, length, ly, decoder, PHONEMES)

            torch.nn.utils.clip_grad_norm_(model.parameters(), 0.25)
            batch_bar.set_postfix(
                loss = f"{total_loss/ (batch+1):.4f}",
                lr = f"{optimizer.param_groups[0]['lr']}",
                distance = f"{total_dist/ (batch+1):.4f}"
            )
            batch_bar.update()
    batch_bar.close()
    return total_loss/(batch+1), total_dist/len(val_loader)

In [ ]:
torch.cuda.empty_cache()
epochs = 10
loss = 5
for epoch in range(epochs):
    loss_t = train(epoch, model, train_loader, optimizer, criterion)
    loss_v, dist_v = validate(model, val_loader, criterion)
    scheduler.step(loss_v)
    if loss_v < loss:
      loss = loss_v
      torch.save(model.state_dict(), "model_best.pt")

Train epoch: 2:  73%|███████▎  | 1517/2072 [32:24<11:52,  1.28s/it, loss=0.1868, lr=5e-05]

# Generate Predictions and Submit to Kaggle

In [ ]:
# prediction
def predict(test_loader, model, decoder):
    with torch.no_grad():
        model.eval()
        preds = []
        for i,data in enumerate(test_loader):
            x,lx=data
            x = x.to(device)
            h,lh=model(x,lx)

            beam_results, beam_scores, timesteps, out_lens = decoder.decode(h.cpu().permute(1,0,2), seq_lens=lh.cpu())
            
            batch_size = beam_results.shape[0]#What is the batch size

            for j in range(batch_size): # Loop through each element in the batch

                h_sliced = beam_results[j,0,:out_lens[j][0]]#TODO: Obtain the beam results
                h_string = "".join([LABELS[x] for x in h_sliced])#TODO: Convert the beam results to phonemes
                preds.append(h_string)

            torch.cuda.empty_cache()
    return preds

In [ ]:
model.load_state_dict(torch.load('/content/model_best.pt'))

<All keys matched successfully>

In [ ]:
# decoder_test = CTCBeamDecoder(labels=PHONEMES, beam_width=100, log_probs_input=True)
decoder_test = CTCBeamDecoder(labels=LABELS, num_processes=os.cpu_count, beam_width=10, log_probs_input=True)
preds = predict(test_loader, model, decoder)

In [ ]:
print(len(preds))

In [ ]:
with open('submission.csv', 'w') as f:
  f.write('index,label\n')
  for i in range(len(preds)):
    f.write(str(i) + ',' + preds[i]+"\n")
!kaggle competitions submit -c 11-785-f22-hw3p2 -f submission.csv -m "I made it!"